In [1]:
import folium
from folium.plugins import MarkerCluster
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import os
from src.category_utils import CATIDX_2_FR_CATNAME
from src.db.baserow_db import DetectedDechet, DechetsTable as DB_manager, TABLE_ID_DEMO, TABLE_ID_REAL
import base64


## Générer des points autour d'un point de base

In [2]:

# Après avoir choisie une liste de point (exemple [(41.2564, 1.5), ...] )
base_points = np.array([(48.99983156103193, 1.6841176691599162), (49.00372755154292, 1.7033343932870884),
                        (48.987855683751064, 1.7193594554471827)])

print(base_points)


def display_map(points, red_points=None):
    m = folium.Map(location=(49.00372755154292, 1.7033343932870884), zoom_start=12)

    for point in points:
        folium.Marker(point).add_to(m)

    # red point [1.6919782206738192, 48.99566143128049]
    if red_points is not None:
        for red_point in red_points:
            folium.Marker(red_point, icon=folium.Icon(color='red')).add_to(m)
    return m


display_map(base_points)


[[48.99983156  1.68411767]
 [49.00372755  1.70333439]
 [48.98785568  1.71935946]]


In [3]:
len(np.array(base_points))

3

In [4]:
def generate_points_around_base_point(base_points: list, n_max=50, distance_max=0.01):
    base_points = np.array(base_points)
    reps = n_max // len(base_points)
    points = np.tile(base_points, (reps, 1))
    points += np.random.uniform(-distance_max, distance_max, points.shape)

    # concat and trim
    return np.concatenate([base_points, points])[0:n_max]


points = generate_points_around_base_point(base_points, n_max=75, distance_max=0.01)
points

array([[48.99983156,  1.68411767],
       [49.00372755,  1.70333439],
       [48.98785568,  1.71935946],
       [49.00500672,  1.67616669],
       [49.00688438,  1.7120198 ],
       [48.9876648 ,  1.72419847],
       [49.00627695,  1.68095344],
       [48.99606705,  1.71070369],
       [48.98469885,  1.7225492 ],
       [49.00821955,  1.69050237],
       [49.01301377,  1.7125959 ],
       [48.98746356,  1.72786137],
       [48.99686386,  1.68755414],
       [48.99868232,  1.69350193],
       [48.97799701,  1.71885906],
       [49.0082126 ,  1.69289149],
       [48.99459129,  1.71046239],
       [48.98616726,  1.72834574],
       [49.00441489,  1.69340886],
       [48.99799851,  1.70305774],
       [48.99263696,  1.71130398],
       [49.00130888,  1.67517804],
       [49.00870883,  1.6979176 ],
       [48.99439729,  1.72196393],
       [48.9991428 ,  1.67982474],
       [49.00133166,  1.71269371],
       [48.97808231,  1.72568937],
       [49.00026266,  1.67506491],
       [49.01232194,

In [5]:
display_map(points, red_points=base_points)

## Ajouter des déchets dans la base de donnée

In [6]:
def load_data(table_id=TABLE_ID_DEMO):
    db_manager = DB_manager(table_id=table_id)
    data = db_manager.get_list_all_rows()

    return pd.json_normalize(data), db_manager


df_data, dechets_db_manager = load_data(TABLE_ID_DEMO)
df_data

,id,order,name,photo,cat_idxs,latitude,longitude,capture_date,processing_date,location,description,uploaded_by,cat_idx_occurences,status.id,status.value,status.color
0,291,1.00000000000000000000,12b3d8a2-2d23-4de9-8011-035adb7e0c53,[{'url': 'https://baserow-media.ams3.digitaloc...,[],48.99983156103193,1.6841176691599162,2024-01-24T21:19:01.536218Z,None,"[48.99983156103193, 1.6841176691599162]",None,[],"{""65"": 3, ""4"": 6, ""64"": 4}",1274759,✅ Déchet Ramassé,green
1,292,2.00000000000000000000,a239f32d-06e3-4888-b001-7a88ed942e70,[{'url': 'https://baserow-media.ams3.digitaloc...,[],49.00372755154292,1.7033343932870884,2024-01-24T21:19:03.111741Z,None,"[49.00372755154292, 1.7033343932870884]",None,[],"{""48"": 4, ""24"": 1}",1274759,✅ Déchet Ramassé,green
2,293,3.00000000000000000000,a6fe9986-f627-40a1-9c6f-164712437f7a,[{'url': 'https://baserow-media.ams3.digitaloc...,[],48.987855683751064,1.7193594554471827,2024-01-24T21:19:04.615514Z,None,"[48.987855683751064, 1.7193594554471827]",None,[],"{""12"": 5}",1274757,✔️ Déchet Confirmé,dark-orange
3,294,4.00000000000000000000,0d81989a-ad1b-4675-9d1b-9ff5fa576c40,[{'url': 'https://baserow-media.ams3.digitaloc...,[],48.991733157754254,1.6849784388309088,2024-01-24T21:19:06.282029Z,None,"[48.991733157754254, 1.6849784388309088]",None,[],"{""53"": 5, ""69"": 1, ""34"": 4, ""54"": 6, ""23"": 6, ...",1274757,✔️ Déchet Confirmé,dark-orange
4,295,5.00000000000000000000,e12dc50b-11b4-4a61-892f-345dc696ff02,[{'url': 'https://baserow-media.ams3.digitaloc...,[],49.012860193827656,1.6950186623034529,2024-01-24T21:19:07.757227Z,None,"[49.012860193827656, 1.6950186623034529]",None,[],"{""80"": 4, ""74"": 2, ""52"": 5, ""77"": 4, ""34"": 6, ...",1274757,✔️ Déchet Confirmé,dark-orange
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,386,96.00000000000000000000,1f36e962-56d7-4271-9a82-034182277ec9,[{'url': 'https://baserow-media.ams3.digitaloc...,[],48.97830006839737,1.7094059033556914,2024-01-24T21:21:27.340554Z,None,"[48.97830006839737, 1.7094059033556914]",None,[],"{""35"": 2}",1274756,📍 Déchet Détecté,orange
96,387,97.00000000000000000000,4a3e5c90-8046-49c4-a267-4d9a68a552dc,[{'url': 'https://baserow-media.ams3.digitaloc...,[],48.99093910367778,1.6809235152498974,2024-01-24T21:21:28.969918Z,None,"[48.99093910367778, 1.6809235152498974]",None,[],"{""78"": 4, ""38"": 3, ""4"": 2, ""81"": 3}",1274759,✅ Déchet Ramassé,green
97,388,98.00000000000000000000,af39109a-85da-44fc-bc54-b65210bb0866,[{'url': 'https://baserow-media.ams3.digitaloc...,[],49.00591158174918,1.7029256686200613,2024-01-24T21:21:30.415052Z,None,"[49.00591158174918, 1.7029256686200613]",None,[],"{""36"": 3, ""43"": 5}",1274756,📍 Déchet Détecté,orange
98,389,99.00000000000000000000,8c1b8f76-8bd2-4ca6-9288-2da44ae97c7a,[{'url': 'https://baserow-media.ams3.digitaloc...,[],48.99258676389096,1.7189189492484942,2024-01-24T21:21:31.749488Z,None,"[48.99258676389096, 1.7189189492484942]",None,[],"{""81"": 5, ""69"": 5, ""4"": 3}",1274759,✅ Déchet Ramassé,green


In [7]:
list_status_id = list(dechets_db_manager.status_options_id_dict.keys())
dechets_db_manager.status_options_id_dict

{1274756: '📍 Déchet Détecté',
 1274757: '✔️ Déchet Confirmé',
 1274758: '📅 Ramassage Planifié',
 1274759: '✅ Déchet Ramassé',
 1274760: '❌ Faux Positif'}

## Générer la fausse data complète

In [9]:
possibles_images: list = glob("../assets/fake_data_photo/*.jpg")

king_dechets = {36: 1.5, 37 : 0.0001, 
                53 : 0.005, 54:0.01,
                59: 0.1, 60: 2, 61:1.2, 62: 0.3, 63: 0.1, 64: 0.6, 65: 0.6, 66: 0.05,
                67: 0.1, 68: 0.2, 69: 0.3, 70: 0.1, 71: 0.2, 72: 0.5, 73: 0.4, 74: 0.2, 75: 0.1, 76: 0.1, 77: 0.5,
                80: 0.5
                }

possibles_cat_idxs = {cat_idx: 1 if cat_idx not in king_dechets.keys() else king_dechets[cat_idx] for cat_idx in
                      CATIDX_2_FR_CATNAME.keys()}
possibles_cat_idxs = {cat_idx: proba / sum(possibles_cat_idxs.values()) for cat_idx, proba in
                      possibles_cat_idxs.items()}  # id and proba

possibles_status: dict = {list_status_id[0]: 0.7, list_status_id[1]: 0.0, list_status_id[2]: 0.25,
                          list_status_id[3]: 0.05, list_status_id[4]: 0.0}  # id and proba 
possibles_status = {status: proba / sum(possibles_status.values()) for status, proba in
                    possibles_status.items()}  # id and proba


In [10]:
from tqdm import tqdm


def init_fake_data(base_points, possibles_images: dict, possibles_cat_idxs: dict, possibles_status: dict, n_points=75,
                   n_cat_name_max_per_point=8, n_occurence_max_per_cat=6,
                   table_id=TABLE_ID_DEMO
                   ):
    print(">>> Let's generate fake data")
    print(f"* table_id : {table_id}")
    print(f"* base_points : {base_points}")
    print(f"* n_points : {n_points}")
    print(f"* n_cat_name_max_per_point : {n_cat_name_max_per_point}")
    print(f"* n_occurence_max_per_cat : {n_occurence_max_per_cat}")

    points = generate_points_around_base_point(base_points, n_max=n_points, distance_max=0.01)
    # delete all rows
    dechets_db_manager = DB_manager(table_id=table_id)
    dechets_db_manager.delete_all_rows(confirmation=True)
    # Pour chaque point, on va générer un faux déchet
    for point in tqdm(list(points), desc=f"Generating fake data (n={n_points}) [{table_id}]"):
        latitude, longitude = point
        image_path = np.random.choice(possibles_images)
        cat_idxs = random.choices(list(possibles_cat_idxs.keys()), weights=list(possibles_cat_idxs.values()),
                                  k=np.random.randint(1, n_cat_name_max_per_point))

        cat_idx_occurences = {cat_idx: random.randint(1, n_occurence_max_per_cat) for cat_idx in cat_idxs}
        status = np.random.choice(list(possibles_status.keys()), p=list(possibles_status.values()))

        r_added = dechets_db_manager.add_dechet_row(image=image_path,
                                                     cat_idx_occurences=cat_idx_occurences,
                                                     longitude=str(longitude),
                                                     latitude=str(latitude),
                                                     status=status,
                                                     generate_description=False,
                                                     verbose=False,
                                                     )
    print("Done")
    return points


init_fake_data(base_points, possibles_images, possibles_cat_idxs, possibles_status, n_points=160,n_cat_name_max_per_point=8, n_occurence_max_per_cat=7)


>>> Let's generate fake data
* table_id : 246046
* base_points : [[48.99983156  1.68411767]
 [49.00372755  1.70333439]
 [48.98785568  1.71935946]]
* n_points : 150
* n_cat_name_max_per_point : 8
* n_occurence_max_per_cat : 7
Found 100 rows.
Deleted.


Generating fake data (n=150) [246046]: 100%|██████████| 150/150 [06:39<00:00,  2.66s/it]

Done


array([[48.99983156,  1.68411767],
       [49.00372755,  1.70333439],
       [48.98785568,  1.71935946],
       [49.0051825 ,  1.69270549],
       [48.99445375,  1.71116172],
       [48.99247522,  1.70953646],
       [49.00782929,  1.69295158],
       [49.00194033,  1.69821802],
       [48.98342552,  1.72317384],
       [49.00799482,  1.68290746],
       [49.00944542,  1.71322104],
       [48.98129519,  1.72080022],
       [49.00749775,  1.68868841],
       [49.01288877,  1.70882189],
       [48.99696028,  1.72537981],
       [49.00607568,  1.68705885],
       [49.00003938,  1.70995555],
       [48.99464698,  1.72604147],
       [49.00101198,  1.67914087],
       [49.00839629,  1.70200602],
       [48.9789459 ,  1.71376923],
       [48.99354865,  1.67693756],
       [48.99729978,  1.70646464],
       [48.98678287,  1.72577012],
       [48.99085919,  1.6892939 ],
       [48.99442561,  1.70515447],
       [48.99442726,  1.72081597],
       [49.00132815,  1.69026704],
       [48.99885515,

### afficher les nouvelles données ajoutées

In [11]:
df_data, _ = load_data(TABLE_ID_DEMO)
df_data

,id,order,name,photo,cat_idxs,latitude,longitude,capture_date,processing_date,location,description,uploaded_by,cat_idx_occurences,status.id,status.value,status.color
0,391,1.00000000000000000000,a8f8eed3-0047-4421-a955-53c901a10305,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 2, 'value': '1'}, {'id': 36, 'value': ...",48.99983156103193,1.6841176691599162,2024-02-08T02:31:36.886225Z,None,"[48.99983156103193, 1.6841176691599162]",None,[],"{""50"": 6, ""36"": 7, ""44"": 5, ""1"": 4, ""35"": 6}",1274756,📍 Déchet Détecté,orange
1,392,2.00000000000000000000,d76202dd-1d8f-4f64-a424-a4a669f5d849,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 23, 'value': '22'}, {'id': 59, 'value'...",49.00372755154292,1.7033343932870884,2024-02-08T02:31:38.846131Z,None,"[49.00372755154292, 1.7033343932870884]",None,[],"{""58"": 3, ""79"": 4, ""22"": 1}",1274756,📍 Déchet Détecté,orange
2,393,3.00000000000000000000,8350b1f4-ab12-4c86-891a-31b1683540d8,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 35, 'value': '34'}, {'id': 47, 'value'...",48.987855683751064,1.7193594554471827,2024-02-08T02:31:40.929777Z,None,"[48.987855683751064, 1.7193594554471827]",None,[],"{""57"": 4, ""46"": 5, ""34"": 5}",1274756,📍 Déchet Détecté,orange
3,394,4.00000000000000000000,d8dfb34a-aaf0-4983-a1c7-82b51b7e4b38,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 8, 'value': '7'}, {'id': 20, 'value': ...",49.00518249950187,1.692705486027613,2024-02-08T02:31:42.523243Z,None,"[49.00518249950187, 1.692705486027613]",None,[],"{""33"": 1, ""19"": 7, ""66"": 6, ""60"": 6, ""7"": 6, ""...",1274756,📍 Déchet Détecté,orange
4,395,5.00000000000000000000,a41c9574-43bd-4994-8a89-055eac39ff6e,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 2, 'value': '1'}, {'id': 4, 'value': '...",48.994453745950175,1.7111617211909618,2024-02-08T02:31:44.243276Z,None,"[48.994453745950175, 1.7111617211909618]",None,[],"{""30"": 3, ""23"": 1, ""33"": 7, ""9"": 3, ""1"": 1, ""3...",1274756,📍 Déchet Détecté,orange
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,536,146.00000000000000000000,4028ac37-7531-40aa-b370-3928c66b2bbd,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 2, 'value': '1'}, {'id': 6, 'value': '...",49.00025515263553,1.7062718651114095,2024-02-08T02:38:04.473087Z,None,"[49.00025515263553, 1.7062718651114095]",None,[],"{""46"": 5, ""1"": 3, ""5"": 5}",1274759,✅ Déchet Ramassé,green
146,537,147.00000000000000000000,a248b914-df94-4d6e-8daa-1a4b1109591d,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 1, 'value': '0'}, {'id': 35, 'value': ...",48.99001415544647,1.7152129994461018,2024-02-08T02:38:07.026852Z,None,"[48.99001415544647, 1.7152129994461018]",None,[],"{""81"": 5, ""0"": 7, ""34"": 7}",1274756,📍 Déchet Détecté,orange
147,538,148.00000000000000000000,21cc7bb2-fa09-4df7-9b42-f069b71b0470,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 21, 'value': '20'}]",48.99019972760179,1.685703582141655,2024-02-08T02:38:08.825942Z,None,"[48.99019972760179, 1.685703582141655]",None,[],"{""20"": 3}",1274756,📍 Déchet Détecté,orange
148,539,149.00000000000000000000,ebba179c-68ee-4fe3-8559-c0baea252565,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 3, 'value': '2'}, {'id': 27, 'value': ...",49.004882930545755,1.696831427384268,2024-02-08T02:38:10.826963Z,None,"[49.004882930545755, 1.696831427384268]",None,[],"{""55"": 1, ""70"": 3, ""26"": 5, ""46"": 6, ""2"": 4}",1274756,📍 Déchet Détecté,orange


In [12]:
points = df_data[['latitude', 'longitude']].values
display_map(points)

In [24]:
m = folium.Map(location=[48.9907, 1.7102], zoom_start=13)

marker_cluster = MarkerCluster().add_to(m)

for i, row_item in df_data.iterrows():
    id_ = row_item['id']
    lat = row_item['latitude']
    long = row_item['longitude']
    print(lat, long)
    photo_url = row_item['photo'][0]['url'] if len(row_item['photo']) > 0 else None
    date = row_item['capture_date']
    status = row_item['status.value']
    status_color = row_item['status.color']
    description = row_item['description']
    if 'cat_idx_occurences' in row_item:
        try:

            cat_idx_occurences = eval(row_item['cat_idx_occurences'])
        except:
            cat_idx_occurences = {}
        else:
            cat_idx_occurences = {int(k): int(v) for k, v in cat_idx_occurences.items()}

    sum_cat_idx_occurences = sum(cat_idx_occurences.values())
    if lat is None or long is None:
        continue
    h4_text = f"{sum_cat_idx_occurences} déchet(s) détecté(s)"

    # crop image to square, 150px
    popup_html = f"""<div>
                            <img src="{photo_url}" width="150" height="100" style="display:block;margin:auto;"><br>
                            <b>Date de capture:</b> {date}<br>
                            <b>Status:</b> {status}<br>
                            <b>Description:</b> {description}
                        </div>
                    """

    iframe = folium.IFrame(popup_html, width=200, height=250)
    #popup = folium.Popup(iframe, max_width=300)
    icon = folium.Icon(color=status_color, icon='trash')
    folium.Marker([lat, long]).add_to(marker_cluster)
    if i > 20:
        break

m

49.0038842828264 1.6840392821199135
49.012331760973225 1.7008874847627227
48.984377895216895 1.7203930565672996
48.990766836328184 1.6765977325127834
49.0122040151781 1.6953912800951436
48.99213579922791 1.7273191024775363
49.001902972259295 1.6780466396762945
49.008733758037096 1.7122044259444105
48.978129928257964 1.728921980091218
49.00830271307917 1.6855142830375245
49.010781100744175 1.6987582117113291
48.98061720924831 1.7124753613329378
49.00017483533106 1.6894220951698748
49.00149844354568 1.7095323994020926
48.97907866681826 1.7135816529316044


C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_2580\1638419640.py:40: UserWarning: color argument of Icon should be one of: {'purple', 'black', 'pink', 'darkgreen', 'beige', 'orange', 'darkpurple', 'darkblue', 'lightgray', 'lightblue', 'blue', 'lightgreen', 'red', 'lightred', 'cadetblue', 'darkred', 'white', 'gray', 'green'}.
  icon = folium.Icon(color=status_color, icon='trash')


In [27]:
df_data_real = load_data(TABLE_ID_REAL)
# long lat to float 
df_data_real['longitude'] = df_data_real['longitude'].astype(float)
df_data_real['latitude'] = df_data_real['latitude'].astype(float)
df_data_real

,id,order,name,photo,cat_idxs,latitude,longitude,capture_date,processing_date,location,description,uploaded_by,cat_idx_occurences,status.id,status.value,status.color
0,66,1.00000000000000000000,5a6c1df8-72e6-4244-9591-d4c7b8495858,[{'url': 'https://baserow-media.ams3.digitaloc...,[],1.674655,48.990726,2024-01-24T10:09:14.520772Z,None,"[1.6746545558850943, 48.99072638869668]",None,[],"{""68"": 3}",1266082,📍 Déchet Détecté,orange
1,67,2.00000000000000000000,e6068891-4f9c-403e-b9ee-3d025787c78e,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 7, 'value': '6'}]",1.697409,49.013100,2024-01-24T10:09:16.775595Z,None,"[1.697408862262514, 49.0130995585845]",None,[],"{""6"": 4, ""81"": 1}",1267691,📅 Ramassage Planifié,light-pink
2,68,3.00000000000000000000,d418863e-cbb5-4805-89bf-1e678a0b1121,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 41, 'value': '40'}, {'id': 50, 'value'...",1.726495,48.986095,2024-01-24T10:09:18.721834Z,None,"[1.7264952512610603, 48.98609506624167]",None,[],"{""62"": 3, ""40"": 4, ""49"": 5, ""64"": 3}",1266081,✅ Déchet Ramassé,green
3,69,4.00000000000000000000,8d8d4a90-aece-41fe-9fff-f6a910b8c6ee,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 22, 'value': '21'}, {'id': 29, 'value'...",1.690684,48.992280,2024-01-24T10:09:20.549467Z,None,"[1.6906839372447227, 48.99228034224054]",None,[],"{""21"": 2, ""28"": 4, ""42"": 4}",1267691,📅 Ramassage Planifié,light-pink
4,70,5.00000000000000000000,f054a1ac-976e-4f87-9871-96785b57f6f7,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 20, 'value': '19'}, {'id': 24, 'value'...",1.707056,49.007635,2024-01-24T10:09:22.222185Z,None,"[1.7070559481233003, 49.00763526562619]",None,[],"{""19"": 4, ""48"": 4, ""23"": 2, ""64"": 5}",1266081,✅ Déchet Ramassé,green
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,175,110.00000000000000000000,f2046863-3509-4588-9b71-1913c948ff06,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 8, 'value': '7'}, {'id': 9, 'value': '...",1.691978,48.995661,2024-01-24T14:22:37.847251Z,None,"[1.6919777043702737, 48.9956606661885]",None,[],"{""7"": 1, ""8"": 1}",1266082,📍 Déchet Détecté,orange
110,176,111.00000000000000000000,476fa07c-2be9-4af0-93af-0df72c08ecbe,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 8, 'value': '7'}, {'id': 9, 'value': '...",1.691979,48.995663,2024-01-24T14:23:06.797523Z,None,"[1.691979030192789, 48.995663039918014]",None,[],"{""7"": 1, ""8"": 1}",1266082,📍 Déchet Détecté,orange
111,177,112.00000000000000000000,dab1e583-0af7-4c4e-94d4-5ad4b65f5dbf,[{'url': 'https://baserow-media.ams3.digitaloc...,[],1.691979,48.995661,2024-01-24T14:30:58.040632Z,None,"[1.6919790587217478, 48.99566063007205]",None,[],"{""60"": 2}",1266082,📍 Déchet Détecté,orange
112,178,113.00000000000000000000,5f83f9fc-82e4-4155-b95a-054c1e2ca601,[{'url': 'https://baserow-media.ams3.digitaloc...,[],48.995667,1.692020,2024-01-24T14:36:22.298182Z,None,"[48.9956669835525, 1.6920204700629506]",None,[],"{""60"": 1}",1266082,📍 Déchet Détecté,orange


In [29]:
# get all rows with wrons long lat values (if long > lat)
df_data_real_wrong_long_lat = df_data_real[df_data_real['longitude'] > df_data_real['latitude']]
df_data_real_wrong_long_lat

,id,order,name,photo,cat_idxs,latitude,longitude,capture_date,processing_date,location,description,uploaded_by,cat_idx_occurences,status.id,status.value,status.color
0,66,1.00000000000000000000,5a6c1df8-72e6-4244-9591-d4c7b8495858,[{'url': 'https://baserow-media.ams3.digitaloc...,[],1.674655,48.990726,2024-01-24T10:09:14.520772Z,None,"[1.6746545558850943, 48.99072638869668]",None,[],"{""68"": 3}",1266082,📍 Déchet Détecté,orange
1,67,2.00000000000000000000,e6068891-4f9c-403e-b9ee-3d025787c78e,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 7, 'value': '6'}]",1.697409,49.013100,2024-01-24T10:09:16.775595Z,None,"[1.697408862262514, 49.0130995585845]",None,[],"{""6"": 4, ""81"": 1}",1267691,📅 Ramassage Planifié,light-pink
2,68,3.00000000000000000000,d418863e-cbb5-4805-89bf-1e678a0b1121,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 41, 'value': '40'}, {'id': 50, 'value'...",1.726495,48.986095,2024-01-24T10:09:18.721834Z,None,"[1.7264952512610603, 48.98609506624167]",None,[],"{""62"": 3, ""40"": 4, ""49"": 5, ""64"": 3}",1266081,✅ Déchet Ramassé,green
3,69,4.00000000000000000000,8d8d4a90-aece-41fe-9fff-f6a910b8c6ee,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 22, 'value': '21'}, {'id': 29, 'value'...",1.690684,48.992280,2024-01-24T10:09:20.549467Z,None,"[1.6906839372447227, 48.99228034224054]",None,[],"{""21"": 2, ""28"": 4, ""42"": 4}",1267691,📅 Ramassage Planifié,light-pink
4,70,5.00000000000000000000,f054a1ac-976e-4f87-9871-96785b57f6f7,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 20, 'value': '19'}, {'id': 24, 'value'...",1.707056,49.007635,2024-01-24T10:09:22.222185Z,None,"[1.7070559481233003, 49.00763526562619]",None,[],"{""19"": 4, ""48"": 4, ""23"": 2, ""64"": 5}",1266081,✅ Déchet Ramassé,green
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,173,108.00000000000000000000,37e17279-af3b-4ba1-ae7a-4e4c9664e924,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 18, 'value': '17'}]",1.691978,48.995663,2024-01-24T14:21:02.633810Z,None,"[1.6919783436638178, 48.99566293835541]",None,[],"{""17"": 5, ""61"": 1, ""72"": 1}",1266082,📍 Déchet Détecté,orange
108,174,109.00000000000000000000,7e64dda8-4809-490f-9d4d-5bdc8cb997ed,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 18, 'value': '17'}, {'id': 39, 'value'...",1.691976,48.995662,2024-01-24T14:22:05.865995Z,None,"[1.69197603294245, 48.995662195688496]",None,[],"{""17"": 3, ""38"": 2, ""74"": 2, ""77"": 2}",1266082,📍 Déchet Détecté,orange
109,175,110.00000000000000000000,f2046863-3509-4588-9b71-1913c948ff06,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 8, 'value': '7'}, {'id': 9, 'value': '...",1.691978,48.995661,2024-01-24T14:22:37.847251Z,None,"[1.6919777043702737, 48.9956606661885]",None,[],"{""7"": 1, ""8"": 1}",1266082,📍 Déchet Détecté,orange
110,176,111.00000000000000000000,476fa07c-2be9-4af0-93af-0df72c08ecbe,[{'url': 'https://baserow-media.ams3.digitaloc...,"[{'id': 8, 'value': '7'}, {'id': 9, 'value': '...",1.691979,48.995663,2024-01-24T14:23:06.797523Z,None,"[1.691979030192789, 48.995663039918014]",None,[],"{""7"": 1, ""8"": 1}",1266082,📍 Déchet Détecté,orange


In [31]:
ids_wrong_long_lat = df_data_real_wrong_long_lat['id'].values
ids_wrong_long_lat

array([ 66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177], dtype=int64)

In [1]:
# change lat <-> long    in the db 
for i, row_item in df_data_real_wrong_long_lat.iterrows():
    id_ = row_item['id']
    lat = row_item['latitude']
    long = row_item['longitude']
    if lat < long:
        dechets_db_manager.update_row(id_=id_, data={'latitude': long, 'longitude': lat})
    

NameError: name 'df_data_real_wrong_long_lat' is not defined